In [1]:
import partridge as ptg
import datetime
import geopandas as gpd
from shapely.geometry import Point
import osmnx as ox
import networkx as nx
import pandas as pd
import os
from shapely.geometry import mapping, shape
from shapely.ops import cascaded_union
from fiona import collection
from IPython.display import clear_output
import threading
import multiprocessing
%matplotlib inline

In [2]:
path=r"/Users/brianparker/Work/VRT_Transit1.zip"

In [3]:
service_ids_by_date=ptg.read_service_ids_by_date(path)

In [4]:
date=datetime.date(2018, 6, 4)

In [ ]:
service_ids=service_ids_by_date[date]

In [ ]:
feed=ptg.load_feed(path,view={
    'trips.txt':{
        'service_id':service_ids,
    },
})

In [ ]:
df=feed.stops

In [ ]:
df['Coordinates']=list(zip(df.stop_lat,df.stop_lon))

In [ ]:
outfolder=r"/Users/brianparker/Work/Walksheds/"

In [ ]:
allwalksheds=[]
allwalksheds=gpd.GeoDataFrame(allwalksheds)
d={}

In [ ]:
def reader2(df):
    for index, row in df.iterrows():
        try:
            clear_output(wait=True)
            stopname=row['stop_name']+"_"+row['stop_id']
            d['{}'.format(stopname)]=ox.graph_to_gdfs((ox.graph_from_point(row['Coordinates'],distance=800,network_type='walk',clean_periphery=False)),node_geometry=False,nodes=False)
            print('reading  {} ({} of {})'.format(row['stop_name'],index,len(df)))
        except Exception:
            print(stopname+" ERROR ERROR ERROR ERROR ERROR")
            continue
        

In [ ]:
# def reader():
#     for i in os.listdir(outfolder):
#         d['{}'.format(i)]=gpd.read_file(outfolder+"\\"+i+"\\edges\\edges.shp")
#         print('reading: '+i)
#     for key,value in d.items():
#         allwalksheds.append(value[:])
#         print('appending: '+ key)
#         return allwalksheds
#     allwalksheds.plot()
#     allwalksheds.to_file(r"N:\Planning - New File Structure\GIS\Data\Stops\allwalkshedsQtr.shp")

In [ ]:
# reader()

In [ ]:
# downloadThreads = []             # a list of all the Thread objects
# for i in range(0, 1400, 100):    # loops 14 times, creates 14 threads
#     downloadThread = threading.Thread(target=reader2)
#     downloadThreads.append(downloadThread)
#     downloadThread.start()
# for downloadThread in downloadThreads:
#     downloadThread.join()
# print('Done.')

In [ ]:
reader2(df)

reading  Overland & Owyhee (231 of 791)


In [ ]:
for key,value in d.items():
    allwalksheds=allwalksheds.append(value[:])
    print('appending: '+ key)

In [ ]:
allwalksheds.plot(figsize=(20,6))

In [ ]:
# # create as many processes as there are CPUs on your machine
# num_processes = multiprocessing.cpu_count()

# # calculate the chunk size as an integer
# chunk_size = int(df.shape[0]/num_processes)

# # this solution was reworked from the above link.
# # will work even if the length of the dataframe is not evenly divisible by num_processes
# chunks = [df.ix[df.index[i:i + chunk_size]] for i in range(0, df.shape[0], chunk_size)]

In [ ]:
# pool=multiprocessing.Pool(processes=num_processes)

In [ ]:
# result=pool.map(reader2,chunks)

In [ ]:
# for key,value in d.items():
#     clear_output(wait=True)
#     allwalksheds=allwalksheds.append(value[:])
#     print(key)
#     allwalksheds.plot()

In [ ]:
# processes=[multiprocessing.Process(target=reader2(df))for x in range(8)]

In [ ]:
# for p in processes:
#     p.start()

In [ ]:
# for p in processes:
#     p.join()

In [ ]:
allwalksheds.head()